In [1]:
import os

In [2]:
import git

In [3]:
import collections

In [4]:
import numpy as np

In [5]:
from git import Repo

In [6]:
import itertools

In [7]:
from conda_merge import merge_dependencies, read_file

# List of workflows provided from the app store 

In [8]:
workflow_dict = { 
    "pyiron-workflow-damask": "https://github.com/materialdigital/pyiron-workflow-damask.git",
    "pyiron-workflow-phasediagram": "https://github.com/materialdigital/pyiron-workflow-phasediagram.git",
    "pyiron-workflow-thermalexpansion": "https://github.com/materialdigital/pyiron-workflow-thermalexpansion.git",
    "pyiron-workflow-TEMImageSegmentation": "https://github.com/materialdigital/pyiron-workflow-TEMImageSegmentation.git"
}

# Clone the selected workflows 

In [9]:
selected_workflows_dict = workflow_dict.copy()

In [10]:
working_dir = os.path.abspath(".")
working_dir

'/home/jovyan'

In [11]:
git_lst, git_name_lst = [], []
for k, v in selected_workflows_dict.items():
    git_repo_path = os.path.join(working_dir, k)  
    git_lst.append(git_repo_path)
    git_name_lst.append(k)
    if not os.path.exists(git_repo_path):
        git.Git(working_dir).clone(v)

# Extract the dependencies for each git tag 

In [12]:
repo_dep_dict = {}
for git_repo, name in zip(git_lst, git_name_lst):
    test_repo = Repo(git_repo)
    dep_dict = {}
    for git_tag in test_repo.tags:
        g = git.Git(git_repo)
        g.checkout(git_tag)
        repo_env = read_file(os.path.join(git_repo, "binder", "environment.yml"))
        dep_dict[git_tag.name] = [l.replace(" ", "") for l in repo_env.get('dependencies')]
    repo_dep_dict[name] = dep_dict

In [13]:
tag_lst = list(itertools.product(*[list(repo_dict.keys()) for repo_dict in repo_dep_dict.values()]))
tag_lst

[('v0.0.1', 'v0.0.1', 'v0.0.1', 'v0.0.1'),
 ('v0.0.1', 'v0.0.1', 'v0.0.2', 'v0.0.1')]

In [14]:
diff_set_lst = list(itertools.product(*[list(repo_dict.values()) for repo_dict in repo_dep_dict.values()]))
diff_set_lst

[(['pyiron=0.4.0',
   'lammps=2020.12.24',
   'sqsgenerator=0.0.5',
   'damask=3.0.0=py37h9964bdb_8',
   'vtk=9.0.1',
   'itkwidgets=0.32.0',
   'pyvista=0.27.4',
   'h5py=3.1.0',
   'nglview=2.7.7',
   'seaborn=0.11.1',
   'papermill'],
  ['pyiron=0.4.0',
   'pyiron_atomistics=0.2.2',
   'lammps=2020.12.24',
   'nglview=2.7.7',
   'sqsgenerator=0.0.5',
   'sphinxdft=2.7.0',
   'papermill'],
  ['pyiron=0.3.23', 'lammps=2020.12.24', 'nglview=2.7.7', 'papermill'],
  ['pyiron_base=0.1.44', 'temmeta=0.0.6', 'pystem=0.0.22', 'papermill']),
 (['pyiron=0.4.0',
   'lammps=2020.12.24',
   'sqsgenerator=0.0.5',
   'damask=3.0.0=py37h9964bdb_8',
   'vtk=9.0.1',
   'itkwidgets=0.32.0',
   'pyvista=0.27.4',
   'h5py=3.1.0',
   'nglview=2.7.7',
   'seaborn=0.11.1',
   'papermill'],
  ['pyiron=0.4.0',
   'pyiron_atomistics=0.2.2',
   'lammps=2020.12.24',
   'nglview=2.7.7',
   'sqsgenerator=0.0.5',
   'sphinxdft=2.7.0',
   'papermill'],
  ['pyiron=0.4.0', 'lammps=2020.12.24', 'nglview=2.7.7', 'paperm

# Merge dependencies 

In [15]:
conflict_dep_lst = []
for diff_set in diff_set_lst:
    merg_deps_lst = merge_dependencies(diff_set)
    dep_name_lst, dep_version_lst, dep_build_lst = [], [], []
    for dl in [d.split("=") for d in merg_deps_lst]:
        if len(dl) == 1:
            dl += ["", ""]
        elif len(dl) == 2:
            dl += [""]
        dep_name_lst.append(dl[0])
        dep_version_lst.append(dl[1])
        dep_build_lst.append(dl[2])
    dep_name_lst, dep_version_lst, dep_build_lst = np.array(dep_name_lst), np.array(dep_version_lst), np.array(dep_build_lst)
    dep_duplicates = [item for item, count in collections.Counter(dep_name_lst).items() if count > 1]
    conflict_lst = []
    for dep in dep_duplicates:
        dup_ind = dep_name_lst == dep
        ver_ind = dep_version_lst != ""
        build_ind = dep_build_lst != ""
        dep_red_version_lst = dep_version_lst[dup_ind & ver_ind]
        dep_red_build_lst = dep_version_lst[dup_ind & ver_ind & build_ind]
        if len(set(dep_red_version_lst)) > 1 or len(set(dep_red_build_lst)) > 1:
            conflict_lst.append([dep, list(set(dep_red_version_lst)), list(set(dep_red_build_lst))])
    conflict_dep_lst.append(conflict_lst)

In [16]:
selected_combination_ind = [len(l) for l in conflict_dep_lst].index(0)
selected_combination_ind

1

In [17]:
{n:tag for n, tag in zip(repo_dep_dict.keys(), tag_lst[selected_combination_ind])}

{'pyiron-workflow-damask': 'v0.0.1',
 'pyiron-workflow-phasediagram': 'v0.0.1',
 'pyiron-workflow-thermalexpansion': 'v0.0.2',
 'pyiron-workflow-TEMImageSegmentation': 'v0.0.1'}

In [18]:
conflict_dep_lst

[[['pyiron', ['0.3.23', '0.4.0'], []]], []]